In [ ]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from pathlib import Path
import tensorflow as tf
from watch_recognition.reports import run_on_image_debug, log_scalar_metrics
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
from functools import partial
from watch_recognition.datasets import get_watch_keypoints_dataset, view_image
import segmentation_models as sm
from watch_recognition.models import get_unet_model, IouLoss2

%matplotlib inline

ROOT_DIR = Path("..")
SAVE_DIR = Path("..")
EPOCHS = 100
image_size = (128, 128)
mask_size = image_size

In [ ]:
X, y, _ = load_keypoints_data_as_kp(
    ROOT_DIR / "keypoints/train",
    autorotate=True,
    image_size=image_size,
)
X.shape, y.shape

In [ ]:
X_val, y_val, _ = load_keypoints_data_as_kp(
    ROOT_DIR / "keypoints/validation",
    autorotate=True,
    image_size=image_size,
)
X_val.shape, y_val.shape

In [ ]:
dataset_train = get_watch_keypoints_dataset(X, y, augment=False, image_size=image_size,
                                            mask_size=mask_size)

In [ ]:
view_image(dataset_train)

In [ ]:
dataset_val = get_watch_keypoints_dataset(X_val, y_val, augment=False, image_size=image_size,
                                            mask_size=mask_size)

In [ ]:
view_image(dataset_val)


In [ ]:
model = get_unet_model(
    unet_output_layer=None,
    image_size=image_size,
    n_outputs=3,
    output_activation='sigmoid',
)

In [ ]:
model.summary()

In [ ]:
loss = IouLoss2()
optimizer = tf.keras.optimizers.Adam()
metrics = [sm.metrics.IOUScore(threshold=0.1), sm.metrics.FScore(threshold=0.1)]
TYPE = "keypoint"
MODEL_NAME = "efficientnetb0-unet-sigmoid-128"

In [ ]:
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics,
)

start = datetime.now()

logdir = SAVE_DIR / f"tensorboard_logs/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
print(logdir)
file_writer_distance_metrics_train = tf.summary.create_file_writer(logdir + "/train")
file_writer_distance_metrics_validation = tf.summary.create_file_writer(
    logdir + "/validation"
)

model_path = SAVE_DIR / f"models/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
model.fit(
    dataset_train,
    epochs=EPOCHS,
    validation_data=dataset_val,
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            log_dir=logdir,
            update_freq="epoch",
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=model_path,
            save_weights_only=False,
            monitor="val_loss",
            save_best_only=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.8,
            patience=5,
            min_lr=1e-6,
            cooldown=3,
            verbose=1,
        ),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(
                log_scalar_metrics,
                X=X,
                y=y,
                file_writer=file_writer_distance_metrics_train,
                model=model,
            )
        ),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(
                log_scalar_metrics,
                X=X_val,
                y=y_val,
                file_writer=file_writer_distance_metrics_validation,
                model=model,
            )
        ),
    ],
)
elapsed = (datetime.now() - start).seconds
print(
    f"total training time: {elapsed / 60} minutes, average: {elapsed / 60 / EPOCHS} minutes/epoch"
)

In [ ]:
train_X, train_y = next(iter(dataset_train))
train_X, train_y = train_X.numpy(), train_y.numpy()

run_on_image_debug(model, train_X[0])

In [ ]:
for image in X_val[10:20]:
    run_on_image_debug(model, image)

In [ ]:
print(model_path)
loaded_model = tf.keras.models.load_model(model_path, compile=False)


for image in X[10:20]:
    run_on_image_debug(loaded_model, image)

In [ ]:
for image in X_val[10:20]:
    run_on_image_debug(loaded_model, image)

